<a href="https://colab.research.google.com/github/clam004/notebook_tutorials/blob/main/dialog_prompt_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%%capture
! pip install transformers

In [1]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
NUM_GPUS = torch.cuda.device_count()
print('NUM_GPUS', NUM_GPUS, "device", device)

NUM_GPUS 1 device cuda:0


### GPU acceleration

To give yourself a GPU in colab, go to `Runtime`-->`Change runtime type`

You can confirm this worked because if you run the above cell again `torch.cuda.device_count()` will change from 0 to the number of GPUs PyTorch now recognizes, this would be 0 to 1 in colab. 

In [2]:
prompt_list = [
    "this is a talk between [human] a human child, and [robot] a kind compassionate Artificial Intelligence chatbot.",
    "[human]: hello",
    "[robot]: how are you today",
    "[human]: im well, but how are you?",
    "[robot]:",
]

prompt = '\n'.join(prompt_list)
print(prompt)

this is a talk between [human] a human child, and [robot] a kind compassionate Artificial Intelligence chatbot.
[human]: hello
[robot]: how are you today
[human]: im well, but how are you?
[robot]:


In [8]:
import os
import transformers
print(transformers.__version__)
from transformers import GPT2Tokenizer, GPT2LMHeadModel

pretrained_hf_model = 'gpt2-large' # 'gpt2-xl' # 

tokenizer = GPT2Tokenizer.from_pretrained(
    pretrained_hf_model,
    pad_token='<|endoftext|>',
)

cache_dir = os.path.join(
    "./modelstates/hugface_models/",
    pretrained_hf_model,
)

print("cache_dir=", cache_dir)

model = GPT2LMHeadModel.from_pretrained(
    pretrained_hf_model,
    cache_dir=cache_dir,
)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("num_params", num_params/1e6, "Million Parameters")

4.25.1
cache_dir= ./modelstates/hugface_models/gpt2-large


Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

num_params 774.03008 Million Parameters


In [12]:
print(model.device)
model = model.to(device)
print(model.device)

cpu
cuda:0


In [13]:
prompt_dict = tokenizer(
    prompt,
    return_tensors="pt",
)

prompt_dict

{'input_ids': tensor([[ 5661,   318,   257,  1561,  1022,   685, 10734,    60,   257,  1692,
          1200,    11,   290,   685,   305, 13645,    60,   257,  1611, 32533,
         35941,  9345,  8537, 13645,    13,   198,    58, 10734,  5974, 23748,
           198,    58,   305, 13645,  5974,   703,   389,   345,  1909,   198,
            58, 10734,  5974,   545,   880,    11,   475,   703,   389,   345,
            30,   198,    58,   305, 13645,  5974]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]])}

In [15]:
prompt_ids = prompt_dict.input_ids
prompt_mask = prompt_dict.attention_mask

prompt_ids = prompt_ids.to(model.device)
prompt_mask = prompt_mask.to(model.device)

prompt_len = prompt_ids.shape[1]

output_ids = model.generate(
    prompt_ids,
    attention_mask = prompt_mask,
    max_length=prompt_len+32,
    pad_token_id=tokenizer.eos_token_id,
)

output_ids

tensor([[ 5661,   318,   257,  1561,  1022,   685, 10734,    60,   257,  1692,
          1200,    11,   290,   685,   305, 13645,    60,   257,  1611, 32533,
         35941,  9345,  8537, 13645,    13,   198,    58, 10734,  5974, 23748,
           198,    58,   305, 13645,  5974,   703,   389,   345,  1909,   198,
            58, 10734,  5974,   545,   880,    11,   475,   703,   389,   345,
            30,   198,    58,   305, 13645,  5974,  1312,   716,   845,   880,
            11,   475,  1312,   716,   845, 10032,   198,    58, 10734,  5974,
          1312,   716,   845, 10032,  1165,   198,    58,   305, 13645,  5974,
          1312,   716,   845, 10032,  1165,   198,    58, 10734]],
       device='cuda:0')

In [19]:
generated_texts = tokenizer.batch_decode(output_ids)
generated_texts

['this is a talk between [human] a human child, and [robot] a kind compassionate Artificial Intelligence chatbot.\n[human]: hello\n[robot]: how are you today\n[human]: im well, but how are you?\n[robot]: i am very well, but i am very tired\n[human]: i am very tired too\n[robot]: i am very tired too\n[human']

In [20]:
generated_text = generated_texts[0]
new_text = generated_text[len(prompt):]
new_text 

' i am very well, but i am very tired\n[human]: i am very tired too\n[robot]: i am very tired too\n[human'

In [21]:
reply = new_text[:new_text.index('\n')]
reply

' i am very well, but i am very tired'

now try https://huggingface.co/clam004/emerg-intent-consistent-good-gpt2-xl-v2